In [1]:
# --- Project Setup ---
from setup_notebook import setup_project_root
setup_project_root()

# --- Imports ---
from src.project_config import RAW_DIR, IMAGES_DIR, COLORS_MODELS, PROCESSED_DIR, PROTEIN_IDS_CSV, get_paths_protein, get_paths
from src.statistics.clustering_algorithm import identify_clusters
from src.statistics.statistics_per_2D import compute_2D_statistics
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np